In [1]:
# Cell 1: Imports
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models
from torchvision.models import ResNet50_Weights, VGG19_Weights  # Import weights enums
from PIL import Image
import zipfile

In [2]:
# Cell 2: Define paths based on Kaggle extraction
train_data_path = '/kaggle/input/dance-form-classification/train'
test_data_path = '/kaggle/input/dance-form-classification/test'

In [3]:
  # Cell 3: Define transformations with data augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
])

In [4]:
# Cell 4: Load dataset and define DataLoader
train_dataset = datasets.ImageFolder(root=train_data_path, transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)

In [ ]:
class HybridModel(nn.Module):
    def __init__(self, num_classes):
        super(HybridModel, self).__init__()

        # Load pre-trained ResNet50 and VGG19 models
        self.resnet50 = models.resnet50(weights=ResNet50_Weights.DEFAULT)
        self.vgg19 = models.vgg19(weights=VGG19_Weights.DEFAULT)

        self.resnet50 = nn.Sequential(*list(self.resnet50.children())[:-1])  
        self.vgg19 = nn.Sequential(*list(self.vgg19.children())[:-1])    

        # Define a new fully connected layer after concatenation
        # ResNet50's last layer has 2048 output features, VGG19 has 25088 (7*7*512) output features after flattening
        self.fc = nn.Sequential(
            nn.Linear(2048 + 25088, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        resnet_features = self.resnet50(x)
        resnet_features = resnet_features.view(resnet_features.size(0), -1)

        vgg_features = self.vgg19(x)
        vgg_features = vgg_features.view(vgg_features.size(0), -1)

        features = torch.cat((resnet_features, vgg_features), dim=1)
        
        out = self.fc(features)
        return out

# Initialize the model
model = HybridModel(num_classes=len(train_dataset.classes))
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 174MB/s] 
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:02<00:00, 207MB/s]  


In [11]:
def train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to('cuda' if torch.cuda.is_available() else 'cpu'), labels.to('cuda' if torch.cuda.is_available() else 'cpu')

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = correct / total
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

        # Step the scheduler
        scheduler.step()

# Train the model 
train_model(model, train_loader, criterion, optimizer, scheduler, num_epochs=20)


Epoch [1/20], Loss: 0.2302, Accuracy: 0.9316
Epoch [2/20], Loss: 0.2442, Accuracy: 0.9199
Epoch [3/20], Loss: 0.2176, Accuracy: 0.9332
Epoch [4/20], Loss: 0.2523, Accuracy: 0.9215
Epoch [5/20], Loss: 0.2784, Accuracy: 0.8965
Epoch [6/20], Loss: 0.2504, Accuracy: 0.9232
Epoch [7/20], Loss: 0.2303, Accuracy: 0.9182
Epoch [8/20], Loss: 0.2359, Accuracy: 0.9265
Epoch [9/20], Loss: 0.2336, Accuracy: 0.9316
Epoch [10/20], Loss: 0.2347, Accuracy: 0.9282
Epoch [11/20], Loss: 0.2337, Accuracy: 0.9349
Epoch [12/20], Loss: 0.2511, Accuracy: 0.9215
Epoch [13/20], Loss: 0.2165, Accuracy: 0.9349
Epoch [14/20], Loss: 0.2151, Accuracy: 0.9432
Epoch [15/20], Loss: 0.2115, Accuracy: 0.9249
Epoch [16/20], Loss: 0.2387, Accuracy: 0.9249
Epoch [17/20], Loss: 0.2440, Accuracy: 0.9149
Epoch [18/20], Loss: 0.2297, Accuracy: 0.9265
Epoch [19/20], Loss: 0.2551, Accuracy: 0.9165
Epoch [20/20], Loss: 0.2117, Accuracy: 0.9416


In [12]:
# Cell 8: Save the model weights
torch.save(model.state_dict(), '/kaggle/working/dance_classification_model.pth')
print("Model weights saved to '/kaggle/working/dance_classification_model.pth'")

Model weights saved to '/kaggle/working/dance_classification_model.pth'


In [13]:
# Cell 9: Function to test the model on training data with per-class accuracy
def test_model(model, train_data_path):
    model.eval() 
    results = []

    with torch.no_grad():
        for class_name in os.listdir(train_data_path):
            class_dir = os.path.join(train_data_path, class_name)

            if os.path.isdir(class_dir):  
                test_images = [f for f in os.listdir(class_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
                correct_predictions = 0
                total_images = len(test_images)

                for image_name in test_images:
                    image_path = os.path.join(class_dir, image_name)
                    image = Image.open(image_path)
                    image = transform(image).unsqueeze(0).to('cuda' if torch.cuda.is_available() else 'cpu')

                    outputs = model(image)
                    _, predicted = torch.max(outputs, 1)

                    predicted_class = train_dataset.classes[predicted.item()]
                    results.append((image_name, predicted_class, class_name))

                    if predicted_class == class_name:
                        correct_predictions += 1

                accuracy = (correct_predictions / total_images) * 100 if total_images > 0 else 0
                print(f'Subdirectory: {class_name}, Correct Predictions: {correct_predictions}/{total_images}, Accuracy: {accuracy:.2f}%')

    return results

# Test the model on training data
test_results = test_model(model, train_data_path)

# Print the predictions
for image_name, predicted_class, actual_class in test_results:
    print(f'Image: {image_name}, Predicted Class: {predicted_class}, Actual Class: {actual_class}')


Subdirectory: mohiniyattam, Correct Predictions: 69/70, Accuracy: 98.57%
Subdirectory: odissi, Correct Predictions: 70/71, Accuracy: 98.59%
Subdirectory: bharatanatyam, Correct Predictions: 66/73, Accuracy: 90.41%
Subdirectory: kuchipudi, Correct Predictions: 69/76, Accuracy: 90.79%
Subdirectory: kathakali, Correct Predictions: 74/74, Accuracy: 100.00%
Subdirectory: manipuri, Correct Predictions: 81/84, Accuracy: 96.43%
Subdirectory: kathak, Correct Predictions: 73/76, Accuracy: 96.05%
Subdirectory: sattriya, Correct Predictions: 73/75, Accuracy: 97.33%
Image: aug_0_8110.png, Predicted Class: mohiniyattam, Actual Class: mohiniyattam
Image: aug_0_9839.png, Predicted Class: mohiniyattam, Actual Class: mohiniyattam
Image: aug_0_2941.png, Predicted Class: mohiniyattam, Actual Class: mohiniyattam
Image: aug_0_368.png, Predicted Class: mohiniyattam, Actual Class: mohiniyattam
Image: aug_0_8698.png, Predicted Class: mohiniyattam, Actual Class: mohiniyattam
Image: aug_0_1209.png, Predicted Cla

In [16]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, message="You are using `torch.load` with `weights_only=False`")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridModel(num_classes=len(train_dataset.classes)) 
model = model.to(device)

# Load the model weights
try:
    model.load_state_dict(torch.load('/kaggle/working/dance_classification_model.pth', map_location=device))
    model.eval()  
    print("Model weights loaded from '/kaggle/working/dance_classification_model.pth'")
except RuntimeError as e:
    print(f"Error loading model weights: {e}")


Model weights loaded from '/kaggle/working/dance_classification_model.pth'


In [17]:
# Cell 11: Function to test a single image
def test_single_image(model, image_path):
    model.eval()  
    with torch.no_grad():
        image = Image.open(image_path)
        image = transform(image).unsqueeze(0).to('cuda' if torch.cuda.is_available() else 'cpu')

        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        class_name = train_dataset.classes[predicted.item()]

    return os.path.basename(image_path), class_name

# Test a single image
image_path_to_test = '/kaggle/input/dance-form-classification/train/bharatanatyam/aug_0_7437.png' 
image_name, predicted_class = test_single_image(model, image_path_to_test)
print(f'Image: {image_name}, Predicted Class: {predicted_class}')


Image: aug_0_7437.png, Predicted Class: bharatanatyam
